In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
from time import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
from tqdm import tqdm
import pandas as pd
import random
import ast
import math


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/assignment-1-nlp/train.csv') # loading training data
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence'])) # changing data-type of entries from 'str' to 'list'

In [ ]:
df = pd.read_csv('/kaggle/input/assignment-1-nlp/test_small.csv') # loading test data
test_data = {} 
for index, row in tqdm(df.iterrows()):
    test_data[row['id']] = ast.literal_eval(row['untagged_sentence']) # changing data-type of entries from 'str' to 'list'

In [ ]:
def display_data(sentence_index):
    '''
        Input : 'sentence_index' (int) -> index of a sentence in training data
        Output: None
    '''
    sentence = data[sentence_index]
    print("TOKEN -> TAG")
    print('...')
    for token, tag in sentence:
        print(token, '>', tag)
sentence_index = random.choice(range(len(data)))
#display_data(sentence_index)

In [ ]:
# cell to show the frequency of each distinct (slack or native) present in the training data
from collections import Counter
distinct_tags = []
word_tags = []
def store_tags():
    
    global distinct_tags
    global word_tags
    
    for sent in data:
        word_tags.append(('START','START'))
        for words, tag in sent:
            word_tags.extend([(tag, words)])
        word_tags.append(('END','END'))
    
store_tags()
tags=[]
tokens = []
freq = {}
poss_tags = {}
for tag, words in word_tags:
    tags.append(tag)
    tokens.append(words)
    if freq.get(words) is None:
        freq[words] = 1
    else:
        freq[words] += 1
    if poss_tags.get(words) is None:
        poss_tags[words] = []
    else:
        poss_tags[words].append(tag)
distinct_tags=list(set(tags))
distinct_tokens = list(set(tokens))
count_tags = {}
for tag, count in Counter(tags).items():
    count_tags[tag] = count
tag_ind = {}
i = 0
tag_map = {}
for tag in distinct_tags:
    tag_ind[tag] = i
    i += 1
token_ind = {}
for i,tag in enumerate(distinct_tags):
    tag_map[tag] = i
for i in tqdm(range(len(distinct_tokens))):
    token_ind[distinct_tokens[i]] = i
unknown = {}
for tags in distinct_tags:
    unknown[tags] = 0
for key, value in freq.items():
    if value < 10:
        for tags in poss_tags[key]:
            unknown[tags] += 1
total_count = sum(unknown.values())

smoothing_factor = 0.0001

# Calculate the total number of unique elements
total_unique_elements = len(unknown)

# Calculate the smoothed probabilities
smoothed_probs = {}
for key, value in unknown.items():
    smoothed_probs[key] = (value + smoothing_factor) / (total_count + smoothing_factor * total_unique_elements)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.xticks(rotation='vertical')
plt.bar(range(len(count_tags)), list(count_tags.values()), align='center')
plt.xticks(range(len(count_tags)), list(count_tags.keys()))
plt.xlabel('Tag')
plt.ylabel('Count')
plt.show()

In [ ]:
submission = {'id': [], 'tagged_sentence' : []} # dictionary to store tag predictions
# NOTE ---> ensure that tagged_sentence's corresponing 'id' is same as 'id' of corresponding 'untagged_sentence' in training data
def store_submission(sent_id, tagged_sentence):
    
    global submission
    submission['id'].append(sent_id)
    submission['tagged_sentence'].append(tagged_sentence)
    
def clear_submission():
    global submission
    submission = {'id': [], 'tagged_sentence' : []}

In [ ]:
T = len(distinct_tags) #number of states
V = len(distinct_tokens) #vocabulary size

# Calculating Initial A, B and Pi Matrices

In [ ]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, tags): # A_t1,t2 
    count_t1 = np.sum(tags == t1)
    count_t2_t1 = np.sum((tags[:-1] == t1) & (tags[1:] == t2))
    return count_t2_t1,count_t1

tags = np.array([pair[0] for pair in word_tags])

A = np.zeros((T, T), dtype='float32')
for i, t1 in enumerate(list(distinct_tags)):
    for j, t2 in enumerate(list(distinct_tags)): 
        count_t2_t1, count_t1 = t2_given_t1(t2, t1, tags)
        A[i, j] = (0.001+count_t2_t1)/ (0.001*T+count_t1)
        


#tags_df = pd.DataFrame(A, columns=list(distinct_tags), index=list(distinct_tags))
#tags_df

In [ ]:
def word_given_tag(word, tag, tag_word_counts): #emission probability b_tag(word)
    count_tag = count_tags[tag]
    count_w_given_tag = tag_word_counts.get(tag, {}).get(word, 0)
    
#     if(count_w_given_tag == 0):
#         return (0.001+count_tag)/(0.001*V+len(tags))
    return (0.00001+count_w_given_tag)/ (0.00001*V+count_tag)

# Preprocess train_bag to create a dictionary of counts for each (tag, word) combination
tag_word_counts = {}
for pair_tag, word_token in word_tags:
    tag_word_counts.setdefault(pair_tag, {}).setdefault(word_token, 0)
    tag_word_counts[pair_tag][word_token] += 1

# Calculate emission probabilities
B = np.zeros((T, V), dtype='float32')
for i, t1 in enumerate(list(distinct_tags)):
    for j, t2 in enumerate(list(distinct_tokens)):
        B[i, j] = word_given_tag(t2, t1, tag_word_counts)

In [ ]:
#compute pi -> initial tag probability
pi_vector = np.zeros(T, dtype = 'float32')
for i in range(T):
    pi_vector[i] = A[tag_ind["START"],i]
# pi_df = pd.DataFrame(pi_vector, index = list(distinct_tags))
# pi_df
#pi_vector

In [ ]:
#compute phi -> end tag probability
phi_vector = np.zeros(T, dtype = 'float32')
for i in range(T):
    phi_vector[i] = A[i,tag_ind["END"]]

#phi_vector

# EM Algorithm implementation using log- sum trick

In [ ]:
def log_sum_exp(values):
    max_value = np.max(values)
    return max_value + np.log(np.sum(np.exp(values - max_value)))

In [ ]:
def forward_prob(sentence):
    T = len(sentence)
    N = len(distinct_tags)
    alpha = np.zeros((N, T), dtype='float32')
    
    # Initialize alpha matrix for the first word
    if sentence[0] not in token_ind:
        alpha[:, 0] = np.log(pi_vector) + np.log(smoothed_probs[distinct_tags])
    else:
        alpha[:, 0] = np.log(pi_vector) + np.log(B[:, token_ind[sentence[0]]])
    
    # Recursively fill alpha matrix using log-sum-exp trick
    for t in range(1, T):
        if sentence[t] not in token_ind:
            emission_prob = np.log(smoothed_probs[distinct_tags])
        else:
            emission_prob = np.log(B[:, token_ind[sentence[t]]])
        alpha[:, t] = log_sum_exp(alpha[:, t-1] + np.log(A.T)) + emission_prob
    
    end_vector = alpha[:,-1] 
    end_val = log_sum_exp(end_vector)
    
    return np.exp(alpha), np.exp(end_val)

#forward_prob(["This","is","test"])

In [ ]:
def backward_prob(sentence):
    T = len(sentence)
    N = len(distinct_tags)
    beta = np.zeros((N, T), dtype='float32')
    
    # Initialize beta matrix for the last word
    beta[:, -1] = 0
    
    # Recursively fill beta matrix using log probabilities and log-sum-exp trick
    for t in range(T-2, -1, -1):
        for s in range(N):
            if sentence[t+1] not in token_ind:
                cur = log_sum_exp(beta[:, t+1] + np.log(A[s, :]) + np.log(smoothed_probs[distinct_tags]))
            else:
                cur = log_sum_exp(beta[:, t+1] + np.log(A[s, :]) + np.log(B[:, token_ind[sentence[t+1]]]))
            beta[s, t] = cur
   
    if sentence[0] not in token_ind:
        start_state = beta[:, 0] + np.log(smoothed_probs[distinct_tags]) + np.log(pi_vector)
    else:
        start_state = beta[:, 0] + np.log(B[:, token_ind[sentence[0]]]) + np.log(pi_vector)
    start_state_val = log_sum_exp(start_state)
    
    return np.exp(beta), np.exp(start_state_val)

In [ ]:
def xi_prob(sentence, forward, backward, forward_val):
    T = len(sentence)
    N = len(distinct_tags)
    xi_log_probabilities = np.zeros((N, T-1, N), dtype='float32')
    
    for t in range(T-1):
        for i in range(N):
            if sentence[t+1] not in token_ind:
                emission_log_prob = np.log(smoothed_probs[distinct_tags])
            else:
                emission_log_prob = np.log(B[:, token_ind[sentence[t+1]]])
            xi_log_probabilities[i, t, :] = forward[i, t] + backward[:, t+1] + np.log(A[i, :]) + emission_log_prob - forward_val
    
    # Convert log probabilities to regular probabilities
    xi_probabilities = np.exp(xi_log_probabilities)
    
    return xi_probabilities

In [ ]:
def gamma_prob(sentence, forward, backward, forward_val):
    N = len(distinct_tags)
    gamma_log_probabilities = forward + backward - forward_val
    
    # Convert log probabilities to regular probabilities
    gamma_probabilities = np.exp(gamma_log_probabilities)

    # Normalize gamma probabilities
    row_sums = np.sum(gamma_probabilities, axis=1, keepdims=True)
    gamma_probabilities /= row_sums

    return gamma_probabilities

# **HMM Learning**

In [ ]:
for iteration in tqdm(range(100)):
    sentence = data[iteration]
    sent = [token for token, tag in sentence]
    T = len(sentence)
    N = len(distinct_tags)

    fwd_probs, fwd_val = forward_prob(sent)
    bwd_probs, bwd_val = backward_prob(sent)
    xi_probabilities = xi_prob(sent, fwd_probs, bwd_probs, fwd_val)
    gamma_probabilities = gamma_prob(sent, fwd_probs, bwd_probs, fwd_val)

    a = np.zeros((N, N))
    
    # Initialize emission matrix 'b' with correct dimensions
    b = np.zeros((N, V))

    # Update 'A' matrix using xi probabilities
    for t in range(T - 1):
        for i in range(N):
            for j in range(N):
                a[i, j] += xi_probabilities[i, t, j]
    
    # Normalize 'A' matrix
    a /= np.sum(a, axis=1, keepdims=True)
    
    # Update 'B' matrix using gamma probabilities
    for j in range(N):
        for vk in range(T):
            # Ensure that the index is within the valid range
            if vk < V:
                indices = np.where(np.array(sent) == sent[vk])[0]
                numerator_b = np.sum(gamma_probabilities[j, indices])
                denominator_b = np.sum(gamma_probabilities[j, :])

                if denominator_b != 0:
                    b[j, vk] = numerator_b / denominator_b

    b[np.isnan(b)] = 0.001

    transition = a
    emission = b

#     new_fwd_temp, new_fwd_temp_val = forward_prob(sent)
#     print('New forward probability: ', new_fwd_temp_val)
#     diff = np.abs(fwd_val - new_fwd_temp_val)
#     print('Difference in forward probability: ', diff)

# **Rule based tagging for unknown words**

In [ ]:
endings_to_tags = {
    "ing": "VB", "ed": "VB", "es": "VB", "ly": "RB",
    "s": "NN", "ness": "NN", "ment": "NN", "ful": "JJ",
    "ive": "JJ", "able": "JJ", "al": "JJ", "ic": "JJ",
    "er": "NN", "or": "NN", "ion": "NN", "ity": "NN"
}

def assign_pos_tag(word):
    if word.isdigit():
        return "CD"  # Tag words involving a digit as numbers
    elif word.istitle():
        return "NN"  # Tag capitalized words as singular proper nouns
    else:
        # Check if the word ends with any of the defined endings
        for ending, tag in endings_to_tags.items():
            if word.endswith(ending):
                return tag
        # If none of the above conditions match, label as singular or mass noun
        return "NN"

# **Viterbi Algorithm using DP**

In [ ]:
def Viterbi(words, A, B):
    T = len(words)
    N = len(distinct_tags)
    
    viterbi = np.zeros((N, T), dtype='float32')
    backpointer = np.zeros((N, T), dtype=int)
    
    # Initialize viterbi matrix for the first word
    for s in range(N):
        if words[0] not in token_ind:
            tag = assign_pos_tag(words[0])
            if(distinct_tags[s]!=tag):
                viterbi[s][0] = math.log(pi_vector[s]) + math.log(0.0001)
            else:
                viterbi[s][0] = math.log(pi_vector[s]) + math.log(0.9999)
        else:
            emission_prob = math.log(B[s, token_ind[words[0]]]) if B[s, token_ind[words[0]]] != 0 else float('-inf')
            viterbi[s][0] = math.log(pi_vector[s]) + emission_prob
            
        backpointer[s][0] = -1
    
    # Recursively fill viterbi matrix
    for t in range(1, T):
        for s in range(N):
            if words[t] not in token_ind:
                tag = assign_pos_tag(words[t])
                emission_prob = math.log(0.9999) if distinct_tags[s] == tag else math.log(0.0001)
            else:
                emission_prob = math.log(B[s, token_ind[words[t]]]) if B[s, token_ind[words[t]]] != 0 else float('-inf')
            
            probabilities = viterbi[:, t-1] + np.log(A[:, s]) + emission_prob
            bestpath = np.argmax(probabilities)
            
            backpointer[s][t] = bestpath
            viterbi[s][t] = probabilities[bestpath]
    
    # Backtrack to find the best tag sequence
    bestpathpointer = np.argmax(viterbi[:, T-1])
    tags = []
    for t in range(T-1, -1, -1):
        if(words[t] not in token_ind):
            tags.append((words[t],assign_pos_tag(words[t])))
        else:
            tags.append((words[t], distinct_tags[bestpathpointer]))
        bestpathpointer = backpointer[bestpathpointer][t]
    
    return tags[::-1]

In [ ]:
#Internal function for calculating average accuracy on the training data itself
def calculate_accuracy(predicted_tags, expected_tags):
    l = len(predicted_tags)
    miss = []
    correct_tags = 0
    for i in range(l):
        if(predicted_tags[i][1]==expected_tags[i][1]):
            correct_tags += 1
        else:
            miss.append((predicted_tags[i][0],predicted_tags[i][1],expected_tags[i][1]))
    correct_tags = sum(1 for pred, exp in zip(predicted_tags, expected_tags) if pred[1] == exp[1])
    #total_tags = len(predicted_tags)
    accuracy = (correct_tags / l) * 100
    return accuracy , miss
total_sum = 0
for i in tqdm(range(0, 100)):
    sentence = data[i]
    untagged_sentence = []
    expected = []
    for token, tag in sentence:
        untagged_sentence.append(token)
        expected.append((token,tag))
#     print("My output: ",Viterbi(untagged_sentence))
#     print("Expected output: ",expected)
    acc, miss = calculate_accuracy(Viterbi(untagged_sentence,A,B),expected)
    total_sum += acc
print("Average accuracy: ",total_sum/100)

In [ ]:
def hmm_tagger_util(sent_id, untagged_sentence):
    if(sent_id in list(submission['id'])):
        return
    tagged_sentence = Viterbi(untagged_sentence,A,B)
    store_submission(sent_id, tagged_sentence)

# MEMM Classifier

In [ ]:
class MEMM:
    
    def __init__(self,tag_map,token_ind,pi_vector):
        self.data = data
        self.tag_map = tag_map
        self.token_ind = token_ind
        self.N = len(tag_map)
        self.V = len(token_ind)
        self.alpha = 0.01
        self.logP = np.empty((1,1))
        self.pi_vector = pi_vector
        
    def construct_features(self):
        word_features = np.zeros((self.N , self.V))
        
        for sent in self.data:
            prev_tag = 'START'
            for i in range(len(sent)):
                word_features[self.tag_map[prev_tag], self.token_ind[sent[i][0]]] += 1
        
        return word_features
                
      
    def train(self):
        word_features = self.construct_features()
        w = np.random.rand(self.N-2, self.N)
    
        for epoch in range(5):
            log_p = np.zeros((self.N-2, self.V))

            for word_ind in range(self.V):
                denom = 0
                for state_ind in range(self.N-2):
                    log_p[state_ind, word_ind] = np.log(np.exp(w[state_ind, :] * word_features[:, word_ind]))
                    denom += np.exp(w[state_ind, :] * word_features[:, word_ind])
                log_p[:, word_ind] -= np.log(denom)

            grad = np.zeros((self.N-2, self.N))

            for word in range(self.V):
                for i in range(self.N-2):
                    for j in range(self.N):
                        grad[i, j] += -(1-np.exp(log_p[i, word])) * word_features[j, word]

            w = w - self.alpha* grad
        
        self.logP = np.zeros((self.N-2, self.V))
        
        for word_ind in range(self.V):
            denom = 0
            for state_ind in range(self.N-2):
                self.logP[state_ind, word_ind] = np.log(np.exp(w[state_ind, :] * word_features[:, word_ind]))
                denom += np.exp(w[state_ind, :] * word_features[:, word_ind])
            self.logP[:, word_ind] -= np.log(denom)
    
    def predict(self , words, distinct_tags):
        T = len(words)
        N = self.N-2

        viterbi = np.zeros((N, T), dtype='float32')
        backpointer = np.zeros((N, T), dtype=int)

        # Initialize viterbi matrix for the first word
        for s in range(N):
            if words[0] not in self.token_ind:
                tag = assign_pos_tag(words[0])
                if(distinct_tags[s]!=tag):
                    viterbi[s][0] = math.log(0.0001)
                else:
                    viterbi[s][0] = math.log(0.9999)
            else:
                emission_prob = self.logP[s, self.token_ind[words[0]]] if self.logP[s, self.token_ind[words[0]]] != 0 else float('-inf')
                viterbi[s][0] = emission_prob

            backpointer[s][0] = -1

        # Recursively fill viterbi matrix
        for t in range(1, T):
            for s in range(N):
                if words[t] not in self.token_ind:
                    tag = assign_pos_tag(words[t])
                    emission_prob = math.log(0.9999) if distinct_tags[s] == tag else math.log(0.0001)
                else:
                    emission_prob = math.logP[s, self.token_ind[words[t]]] if self.logP[s, self.token_ind[words[t]]] != 0 else float('-inf')

                probabilities = viterbi[:, t-1] + emission_prob
                bestpath = np.argmax(probabilities)

                backpointer[s][t] = bestpath
                viterbi[s][t] = probabilities[bestpath]

        # Backtrack to find the best tag sequence
        bestpathpointer = np.argmax(viterbi[:, T-1])
        tags = []
        for t in range(T-1, -1, -1):
            if(words[t] not in self.token_ind):
                tags.append((words[t],assign_pos_tag(words[t])))
            else:
                tags.append((words[t], distinct_tags[bestpathpointer]))
            bestpathpointer = backpointer[bestpathpointer][t]

        return tags[::-1]

In [ ]:
def memm_tagger_util(sent_id, untagged_sentence):
    if(sent_id in list(submission['id'])):
        return
    MEMMClassifier = MEMM(tag_map,token_ind,pi_vector)
    tagged_sentence = MEMMClassifier.predict(pi_vector)
    store_submission(sent_id, tagged_sentence)

In [ ]:
# cell to implement tagger that allots random tags to words in a sentence

def random_tagger_util(sent_id, untagged_sentence):
    if(sent_id in list(submission['id'])):
        return
    tagged_sentence = []
    for word in untagged_sentence:
        tagged_sentence.append((word, random.choice(distinct_tags)))
    store_submission(sent_id, tagged_sentence)
    

In [ ]:
for sent_id in tqdm(list(test_data.keys())):
    sent = test_data[sent_id]
    hmm_tagger_util(sent_id, sent)

In [ ]:
path_to_directory = '/kaggle/working/'
pd.DataFrame(submission).to_csv(path_to_directory +' sample_submission.csv', index = False)